In [ ]:
%pip install python-dotenv -q
%pip install SPARQLWrapper -q
%pip install pandas -q
%pip install neo4j -q

# Exploring data

In [40]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

# P31 instance of
# Q11688446 roman deity
# P2925 domain of saint or deity
sparql.setQuery("""
SELECT ?romainDeity ?romainDeityLabel ?romainDeityDescription (group_concat(?domainOfSaint;separator=";") as ?domainsOfSaint) {
  ?romainDeity wdt:P31 wd:Q11688446;
  wdt:P2925  ?domainOfSaint .
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
GROUP BY ?romainDeity ?romainDeityLabel ?romainDeityDescription
LIMIT 5
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for row in results["results"]["bindings"]:
    # We want the "value" attribute of the "comment" field
    
    title = row["romainDeityLabel"]["value"]
    if ('romainDeityDescription' in row):
      title += "\t" + row['romainDeityDescription']['value']
    
    print(title)    
    domains = row['domainsOfSaint']['value'].split(';')    
    if (domains):
      for domain in domains:
          print("\t" + domain)
    
pass

Mars	Roman god of war, guardian of agriculture
	http://www.wikidata.org/entity/Q198
	http://www.wikidata.org/entity/Q11451
Ceres	Roman goddess of agriculture
	http://www.wikidata.org/entity/Q11451
	http://www.wikidata.org/entity/Q964401
	http://www.wikidata.org/entity/Q1956046
	http://www.wikidata.org/entity/Q2995529
Gaia	Greek primordial deity, the personification of the Earth
	http://www.wikidata.org/entity/Q964401
Juno	ancient Roman goddess
	http://www.wikidata.org/entity/Q8445
	http://www.wikidata.org/entity/Q42008
	http://www.wikidata.org/entity/Q866081
	http://www.wikidata.org/entity/Q14819852
Minerva	Roman goddess of wisdom and sponsor of arts, trade, and defense
	http://www.wikidata.org/entity/Q37226
	http://www.wikidata.org/entity/Q49757
	http://www.wikidata.org/entity/Q1294787
	http://www.wikidata.org/entity/Q43109919


In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

# P2925 domain of saint or deity
sparql.setQuery("""
SELECT DISTINCT ?value ?valueLabel
WHERE 
{
  ?item wdt:P2925 ?value
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
LIMIT 100
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for row in results["results"]["bindings"]:
    # We want the "value" attribute of the "comment" field
    
    title = row['valueLabel']['value']
    print(title)
    

    
pass

In [ ]:
# print(results)

# import pandas as pd

# df = pd.json_normalize(results['results']['bindings'])
# df

# import pandas as pd

# results_df = pd.io.json.json_normalize(results['results']['bindings'])
# results_df[['romainDeityLabel.value', 'romainDeity.value', 'romainDeityDescription.value', 'domainOrSaints.value']]


# df[['romainDeity.value', 'romainDeityLabel.value', 'romainDeityDescription.value']]
# df.sort_values(by=['itemLabel.value'])[['item.value', 'itemLabel.value', 'itemDescription.value']]

# Fetch all war deities


In [142]:

# P31 instance of
# Q41863069 war deity
sparql.setQuery("""
SELECT ?wargod ?wargodLabel (group_concat(?domainOfSaint;separator=";") as ?domainsOfSaint) {
  ?wargod wdt:P31 wd:Q41863069;
  wdt:P2925  ?domainOfSaint
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
GROUP BY ?wargod ?wargodLabel
""")

sparql.setReturnFormat(JSON)
results = sparql.query().convert()

print_limit = 10
for row in results["results"]["bindings"][:print_limit]:
  print(row["wargodLabel"]["value"] + ": " + row["wargod"]['value'])  
  
  domains = row['domainsOfSaint']['value'].split(';')    
  if (domains):
    for domain in domains:
        print(" - " + domain)
  

Mars: http://www.wikidata.org/entity/Q112
 - http://www.wikidata.org/entity/Q198
 - http://www.wikidata.org/entity/Q11451
Ares: http://www.wikidata.org/entity/Q40901
 - http://www.wikidata.org/entity/Q198
Horus: http://www.wikidata.org/entity/Q84122
 - http://www.wikidata.org/entity/Q527
 - http://www.wikidata.org/entity/Q11573140
Minerva: http://www.wikidata.org/entity/Q126916
 - http://www.wikidata.org/entity/Q37226
 - http://www.wikidata.org/entity/Q49757
 - http://www.wikidata.org/entity/Q1294787
 - http://www.wikidata.org/entity/Q43109919
Astarte: http://www.wikidata.org/entity/Q130274
 - http://www.wikidata.org/entity/Q198
 - http://www.wikidata.org/entity/Q964401
 - http://www.wikidata.org/entity/Q4042967
Seth: http://www.wikidata.org/entity/Q131795
 - http://www.wikidata.org/entity/Q198
 - http://www.wikidata.org/entity/Q79871
 - http://www.wikidata.org/entity/Q124490
 - http://www.wikidata.org/entity/Q182878
 - http://www.wikidata.org/entity/Q473741
 - http://www.wikidata.org/

## Neo4j

In [17]:
import os
from neo4j import GraphDatabase
from dotenv import load_dotenv

load_dotenv()
url = os.getenv('NEO4J_BOLT_URL')
user = os.getenv('NEO4J_USERNAME')
password = os.getenv('NEO4J_PASSWORD')

driver = GraphDatabase.driver(url, auth=(user,password))

In [137]:
# Delete all nodes & relationships
with driver.session() as session:
    session.run("MATCH (n) DETACH DELETE n")

In [ ]:
with driver.session() as session:  
  for row in results["results"]["bindings"]:
      # We want the "value" attribute of the "comment" field
      print(row["wargod"]['value'] + "\t" + row["wargodLabel"]["value"])
      
      session.run("CREATE (g:God { name: $name, entity: $entity}) RETURN g", 
                  name=row["wargodLabel"]["value"], 
                  entity=row["wargod"]['value'])
      
      domains = row['domainsOfSaint']['value'].split(';')    
      if (domains):
        for domain in domains:
            print("\t" + domain)
            
            session.run("MERGE (d:Domain { entity: $entity}) RETURN d", entity=domain)
            
              
            session.run("MATCH (g2:God {entity: $god_entity}), (d2:Domain {entity: $domain_entity}) MERGE (g2)-[r:GOD_OF]->(d2)",
                        god_entity=row["wargod"]['value'],
                        domain_entity=domain)